# Redes neuronales para estimar la calidad del concreto

Vamos a crear un modelo de redes neuronales para predecir la fuerza de una preparación de concreto, dada su composición. El diccionario de datos es el siguiente:

- cement (componente 1): cemento, kg en un metro cúbico de mezcla
- slag (componente 2): escorias de horno, kg en un metro cúbico de mezcla
- ash (componente 3): ceniza, kg en un metro cúbico de mezcla
- water (componente 4): agua,  kg en un metro cúbico de mezcla
- superplastic (componente 5): superplastificante, kg en un metro cúbico de mezcla
- coarseagg (componente 6): agregado grueso, kg en un metro cúbico de mezcla
- fineagg (componente 7): agregado fino, kg en un metro cúbico de mezcla
- age: días desde que se creó la mezcla
- strength: fuerza compresiva del concreto, en MPa (**variable objetivo**)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.model_selection import GridSearchCV #permite buscar la mejor configuración de parámetros con C-V
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

## Entendimiento de los datos

<font color = "red">Cargamos los datos y buscamos problemas de calidad de datos.</font>

...

...

...

...

<font color = "red">Preparamos los datos en entrenamiento (X_train, y_train) y de evaluación (X_test, y_test).</font>

...

...

...

...

<font color = "red">Establecemos el baseline de regresión y encontramos su MAE y R2.</font>

...

...

...

...

## Modelamiento

* <font color = "red">Creamos un modelo de regresión **MLPRegressor** con 3 capas de 30 neuronas cada una.</font> 
* <font color = "red">Lo entrenamos con los demás parámetros por defecto sobre el training set y lo evaluamos con el test set, obteniendo el MAE y R2 correspondiente.</font> 
* <font color = "red">Lo comparamos con el modelo baseline.</font>

...

...

...

...

<font color = "red">Encuentre la mejor red neuronal utilizando **GridSearchCV**, buscando la mejor combinación de los parámetros siguientes:</font>
* <font color = "red">**activation**: función de activación, a escoger entre 'logistic', 'tanh', 'relu' (valor por defecto)</font>
* <font color = "red">**max_iter**: máximo número de épocas de entrenamiento (por defecto, 200). Puede que no se necesiten todas las especificadas si se llega a convergencia).</font>
* <font color = "red">**hidden_layer_sizes**: topología de la red, vector indicando el número de neuronas por capa. Por defecto solo se tiene un capa escondidad con 100 neuronas: (100).</font>
* <font color = "red">**learning_rate_init**: tasa de aprendizaje inicial (por defecto es constante aunque se puede modificar esta tasa a medida que se va avanzando en el número de épocas). Por defecto, el valor es 0.001. </font>

In [ ]:
activation_vec = ['logistic', 'relu', 'tanh']
max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), (30, 20, 10)]
learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05]

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(30,30,30))

### activation

In [ ]:
import time
start = time.time() # Devuelve el tiempo actual en segundos desde el 1o de enero de 1970 (punto de referencia)

np.random.seed(1234)
parametros = {'activation': activation_vec
              }
scoring = {'mae':'neg_mean_absolute_error', 'r2':'r2'}
grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='mae', n_jobs=-1, iid=True)
grid.fit(X_train, y_train)

print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un MAE de {1:.2f}% y un R2 de {2:.2f}".format(
    grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_r2'][grid.best_index_]*100))
end = time.time() # Tiempo después de finalizar el entrenamiento del modelo
print("Tiempo total: {0:.2f} minutos".format((end-start)/60))

In [ ]:
df = pd.DataFrame([(activation, mae*100, r2*100) for (activation, mae, r2) in 
                   zip(activation_vec, 
                       grid.cv_results_['mean_test_mae'], 
                       grid.cv_results_['mean_test_r2'],
                      )
                   ], columns = ('activation', 'MAE', 'R2'))

In [ ]:
df

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred), ", R2:", r2_score(y_test, y_pred), "\n")

### max_iter

...

...

...

...

### hidden_layer_sizes

...

...

...

...

### learning_rate_init

...

...

...

...

### mejor combinación

In [93]:
activation_vec = ['logistic', 'relu', 'tanh']
max_iter_vec = [10, 20, 50, 75, 100, 200, 300, 400, 500, 1000, 2000]
hidden_layer_sizes_vec = [(10,), (20,), (30,), (10, 10), (20, 20), (30, 30), (20, 10), 
                          (10, 10, 10), (20, 20, 20), (30, 30, 30), (30, 20, 10)]
learning_rate_init_vec = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]

In [94]:
import time
start = time.time() # Devuelve el tiempo actual en segundos desde el 1o de enero de 1970 (punto de referencia)

np.random.seed(1234)
parametros = {'activation': activation_vec,
              'max_iter':max_iter_vec,
              'hidden_layer_sizes': hidden_layer_sizes_vec,
              'learning_rate_init': learning_rate_init_vec
              }

scoring = {'mae':'neg_mean_absolute_error', 'r2':'r2'}
grid = GridSearchCV(mlp, param_grid=parametros, cv=5, scoring=scoring, refit='mae', n_jobs=-1, iid=True)

### <font color="red"><b>DESDE AQUÍ: NO EJECUTAR DE NUEVO (+20 minutos)</b></font>

In [95]:
grid.fit(X_train, y_train)

print("Los parámetros del mejor modelo fueron {0}, que permiten obtener un MAE de {1:.2f}% y un R2 de {2:.2f}".format(
    grid.best_params_, grid.best_score_*100, grid.cv_results_['mean_test_r2'][grid.best_index_]*100))
end = time.time() # Tiempo después de finalizar el entrenamiento del modelo
print("Tiempo total: {0:.2f} minutos".format((end-start)/60))

Los parámetros del mejor modelo fueron {'activation': 'relu', 'hidden_layer_sizes': (30, 20, 10), 'learning_rate_init': 0.001, 'max_iter': 300}, que permiten obtener un MAE de -550.91% y un R2 de 79.27
Tiempo total: 13.49 minutos


### <font color="red"><b>HASTA ACÁ (+15 minutos)</b></font>

In [96]:
df = pd.DataFrame([(mae*100, r2*100) for (mae, r2) in 
                   zip( 
                       grid.cv_results_['mean_test_mae'], 
                       grid.cv_results_['mean_test_r2'],
                      )
                   ], columns = ('MAE', 'R2'))

In [97]:
df.iloc[np.argsort(-df.MAE),]

,MAE,R2
2547,-550.910383,79.267352
2595,-562.852486,79.331937
2515,-563.556110,79.750992
2481,-566.691616,79.064271
2627,-566.980789,79.421641
...,...,...
1705,-5858.788211,-3237.991388
1453,-6131.078533,-3006.630877
1452,-6490.110760,-3730.478863
1342,-7997.332578,-6234.052438


In [98]:
grid.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_activation', 'param_hidden_layer_sizes', 'param_learning_rate_init', 'param_max_iter', 'params', 'split0_test_mae', 'split1_test_mae', 'split2_test_mae', 'split3_test_mae', 'split4_test_mae', 'mean_test_mae', 'std_test_mae', 'rank_test_mae', 'split0_test_r2', 'split1_test_r2', 'split2_test_r2', 'split3_test_r2', 'split4_test_r2', 'mean_test_r2', 'std_test_r2', 'rank_test_r2'])

In [99]:
y_pred = grid.best_estimator_.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred), ", R2:", r2_score(y_test, y_pred), "\n")

MAE: 6.392217087703705 , R2: 0.7762798515016519 



In [100]:
df = pd.DataFrame([(act, hidden_layers, lr, max_iter, acc*100, kappa*100) for (act, hidden_layers, lr, max_iter, acc, kappa) in 
                   zip(
                       grid.cv_results_['param_activation'], 
                       grid.cv_results_['param_hidden_layer_sizes'], 
                       grid.cv_results_['param_learning_rate_init'], 
                       grid.cv_results_['param_max_iter'], 
                       grid.cv_results_['mean_test_mae'], 
                       grid.cv_results_['mean_test_r2'],
                      )
                   ], columns = ('Activation', 'HiddenLayers', 'LearningRate', 'MaxIter', 'MAE', 'R2'))

In [101]:
df.iloc[np.argsort(-df.MAE),].head(20)

,Activation,HiddenLayers,LearningRate,MaxIter,MAE,R2
2547,relu,"(30, 20, 10)",0.001,300,-550.910383,79.267352
2595,relu,"(30, 20, 10)",0.005,2000,-562.852486,79.331937
2515,relu,"(30, 30, 30)",0.009,400,-563.556110,79.750992
2481,relu,"(30, 30, 30)",0.006,300,-566.691616,79.064271
2627,relu,"(30, 20, 10)",0.008,1000,-566.980789,79.421641
2474,relu,"(30, 30, 30)",0.005,2000,-569.073427,79.724852
2546,relu,"(30, 20, 10)",0.001,200,-571.885470,78.976864
2557,relu,"(30, 20, 10)",0.002,200,-572.794930,78.855431
2551,relu,"(30, 20, 10)",0.001,2000,-575.481511,78.311116
2592,relu,"(30, 20, 10)",0.005,400,-581.176688,78.961343
